In [1]:
#KY-038
import busio
import digitalio
import board
import time
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objects as go
import adafruit_mcp3xxx.mcp3008 as MCP
from adafruit_mcp3xxx.analog_in import AnalogIn
from scipy.signal import savgol_filter


spi = busio.SPI(clock=11, MISO=9, MOSI=10)
cs = digitalio.DigitalInOut(board.D5)
mcp = MCP.MCP3008(spi, cs)

In [2]:
import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BCM)
GPIO.setup(14, GPIO.OUT)
p = GPIO.PWM(14, 1)
p.start(100)

In [3]:
p.start(1)

In [46]:
p.ChangeDutyCycle(1)

In [42]:
fileindex = 0

In [49]:
dc = 100
f = 5
title = f'blink dc={dc}% freq={f} Hz'
tmax = 20 #sec
fileindex += 1
has_v_control = False
p.ChangeDutyCycle(dc)
p.ChangeFrequency(f)
#title = f'blink dc={dc}% freq={f} Hz'
test = []
case = []
nothing = []
x = []
start = datetime.now()
while datetime.now() < start + timedelta(seconds=tmax):
    test.append(AnalogIn(mcp, MCP.P0).voltage)
    if has_v_control:
        case.append(AnalogIn(mcp, MCP.P1).voltage)
    #nothing.append(3.3 - AnalogIn(mcp, MCP.P2).voltage)
    t = datetime.now() - start
    x.append(t.seconds + t.microseconds*1e-6)
print('Done.')
data = [go.Scatter(x=x, y=test, name='Photoresistor', mode='lines')]
if has_v_control:
    data.append(go.Scatter(x=x, y=case, name='Vo', mode='lines'))
fig = go.Figure(data=data,
                layout=dict(title=title, xaxis={'title': 'time [s]'},
                            yaxis={'title': 'Input V [V]'}))
fig.show()
df = pd.DataFrame(list(zip(x, test)), 
               columns =['Time', 'Voltage'])
filename = f'test_{dc}pc_{f}Hz_{tmax}s_{fileindex}.csv'
df.to_csv(filename)
print(filename)

Done.


test_100pc_5Hz_20s_7.csv


In [38]:
df = pd.DataFrame(list(zip(x, test)), 
               columns =['Time', 'Voltage']) 
df.to_csv('long.csv')

In [41]:

fig = go.Figure(data=[go.Scatter(x=x, y=savgol_filter(np.array(test), 11,3), name='Photoresistor')#,
                      #go.Scatter(y=case, name='Vo')#,
                      #go.Scatter(x=x, y=nothing, name='shorting?')
                      ],
                layout=dict(title=title, xaxis={'title': 'time [s]'},
                            yaxis={'title': 'Input V [V]'}))
fig.show()

/home/pi/berryconda3/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning:

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.

